In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/iamdipsan/100-Days-ML/refs/heads/main/day-32/train.csv",usecols=['Age','Fare','Survived'])

In [3]:
df.dropna(inplace=True)

In [4]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [7]:
X=df.drop(columns=['Survived'])
y=df['Survived']

In [8]:
X

,Age,Fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
...,...,...
885,39.0,29.1250
886,27.0,13.0000
887,19.0,30.0000
889,26.0,30.0000


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [11]:
y_pred= clf.predict(X_test)


In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6223776223776224

In [13]:
from sklearn.model_selection import cross_val_score
np.mean(cross_val_score(DecisionTreeClassifier(),X, y, cv=10, scoring='accuracy'))

np.float64(0.6330790297339594)

In [15]:
###Discretization (Binning)
from sklearn.preprocessing import KBinsDiscretizer
kbin_age= KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare=KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [19]:
from sklearn.compose import ColumnTransformer
trf=ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1]),
])

In [20]:
X_train_transformed=trf.fit_transform(X_train)
X_test_transformed=trf.transform(X_test)

In [21]:

trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [22]:

trf.named_transformers_['second'].bin_edges_

array([array([  0.    ,   7.25  ,   7.775 ,   7.8958,   8.1583,  10.5   ,
               13.    ,  14.4542,  18.75  ,  26.    ,  26.55  ,  31.275 ,
               51.4792,  76.2917, 108.9   , 512.3292])                   ],
      dtype=object)

In [26]:
output = pd.DataFrame({
    'age': X_train['Age'],
    'age_trf': X_train_transformed[:, 0],
    'fare': X_train['Fare'],
    'fare_trf': X_train_transformed[:, -1]  # Corrected typo from 'fate_trf'
})

In [28]:
output

,age,age_trf,fare,fare_trf
328,31.0,8.0,20.5250,8.0
73,26.0,6.0,14.4542,7.0
253,30.0,8.0,16.1000,7.0
719,33.0,9.0,7.7750,2.0
666,25.0,6.0,13.0000,6.0
...,...,...,...,...
92,46.0,12.0,61.1750,12.0
134,25.0,6.0,13.0000,6.0
337,41.0,11.0,134.5000,14.0
548,33.0,9.0,20.5250,8.0


In [29]:

output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())

In [30]:

output['age_labels']

,age_labels
328,"(30.0, 32.0]"
73,"(25.0, 28.0]"
253,"(28.0, 30.0]"
719,"(32.0, 35.0]"
666,"(23.0, 25.0]"
...,...
92,"(42.0, 47.0]"
134,"(23.0, 25.0]"
337,"(38.0, 42.0]"
548,"(32.0, 35.0]"


In [31]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
211,35.0,10.0,21.0000,8.0,"(32.0, 35.0]","(18.75, 26.0]"
866,27.0,6.0,13.8583,6.0,"(25.0, 28.0]","(13.0, 14.454]"
523,44.0,12.0,57.9792,12.0,"(42.0, 47.0]","(51.479, 76.292]"
372,19.0,3.0,8.0500,3.0,"(16.0, 19.0]","(7.896, 8.158]"
345,24.0,5.0,13.0000,6.0,"(23.0, 25.0]","(10.5, 13.0]"


In [34]:
clf=DecisionTreeClassifier()
clf.fit(X_train_transformed,y_train)
y_pred2=clf.predict(X_test_transformed)

In [35]:
accuracy_score(y_test,y_pred2)

0.6363636363636364

In [36]:

X_trf=trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))


np.float64(0.6358959311424099)